In [619]:
from ipynb.fs.full.ETDaugmentation import *
from imgaug import augmenters as iaa
import imgaug as ia
import pandas as pd
import numpy as np
import cv2
import glob
import re
import csv

In [341]:
data = pd.read_csv('cv_data.csv')

In [342]:
def cleanResume(resumeText):
    resumeText = re.sub('http\S+\s*', ' ', resumeText)  # remove URLs
    resumeText = re.sub('RT|cc', ' ', resumeText)  # remove RT and cc
    resumeText = re.sub('#\S+', '', resumeText)  # remove hashtags
    #resumeText = re.sub('@\S+', '  ', resumeText)  # remove mentions
    #resumeText = re.sub('[%s]' % re.escape("""!"#$%&'()*+-./;:<=>?@[\]^_`{|}~"""), ' ', resumeText)  # remove punctuations
    resumeText = re.sub(r'[^\x00-\x7f]','', resumeText)
    return resumeText

In [343]:
data['cleaned_text'] = data['Resume'].apply(lambda x: cleanResume(x))
df = data['cleaned_text']
resume_list = df.values.tolist()

In [614]:
W = 2360
H = 3200

def text_on_img(fileName, text, size):
    font = ImageFont.truetype('NimbusMonoPS-Bold.otf', size)
    image = Image.new(mode = "RGB", size = (W,H), color = "white")
    draw = ImageDraw.Draw(image)
    w, h = draw.textsize(text)
    ## Adjust the texual position in a page and draw the text on a image
    '''
    dedication -- 14 width and height 8; 
    ack -- 15 width and 6 height; 
    gabs -- 13 width and height 16
    abstract -- 16 width and height 12
    chapter abstract -- 18 width and height 6
    '''
    alignments = ["left"]
    y = 10
    for alignment in alignments:
        draw.text(((W-1000)/15,(H-600)/16), text, align=alignment, font=font, fill=(0,0,0), spacing=15)
        image.save(fileName)

    return image
    
## This function is calling text_on_image function and save the image to the directory
def cv_aug(cv_text_list):
    for n, row in enumerate(cv_text_list):
        fileName = ("/home/mchou001/Label-CurriculumVitae/aug_text{}.png".format(n))
        save_image = text_on_img(fileName, row, 32) ## dedication -- 43, ack and abstract - 40, gabs -- 38

    return save_image

def img_transform(img_files):

    img_augmentation = iaa.Sequential([
        iaa.Affine(rotate=(-6, 6)),
        iaa.AdditiveGaussianNoise(scale=(10, 50)),
        iaa.SaltAndPepper(p=0.1),
        iaa.GaussianBlur(sigma=0.5),
        iaa.LinearContrast(alpha=1),
        iaa.PerspectiveTransform(scale=0.025, keep_size=True)
    ], random_order = True)

    lower = 0
    upper = 1000
    while lower < upper:
        for n, images in enumerate(img_files[0:]):
            aug_image = img_augmentation(image=cv2.imread(images))
            cv2.imwrite('./Label-CurriculumVitae/506/aug{}.png'.format(lower),aug_image)
            lower += 1

In [615]:
#resume_list[0:4]

In [616]:
text_title = []
resume_text = []
for rows in resume_list[0:]:
    lines = [rows]
    for txt in lines:
        text_wrap = re.sub('(^[a-z]){1,100}\s?', '\n', txt)
        text_wrap = re.sub('\.\s?', '\n', txt)
        text_wrap = re.sub('  ', ' * ', text_wrap)
        text_wrap = re.sub('#\s|/', '\n', text_wrap)
        #text_wrap = re.sub('[^,{1,100}]', '\n', text_wrap)
        text_title.append(text_wrap)

text_titles = "\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\tCURRICULUM VITAE\n\n"
for rows in text_title:
    resume = text_titles + rows
    resume_text.append(resume)

In [620]:
if __name__ == "__main__":
    parser = Preprocessor()
    img_path = glob.glob('./Label-CurriculumVitae/*.png')
    cv_text_aug = cv_aug(resume_text)
    cv_img_aug = img_transform(img_path)

    image_path = sorted(glob.glob('/home/mchou001/Label-CurriculumVitae/506/*.png'), key = parser.numericalSort)
    df = pd.DataFrame(image_path, columns = ['filename'])
    df['class'] = np.nan
    df = df.fillna("Label-CurriculumVitae")
    df.to_csv('cv_img_aug.csv', encoding = 'utf-8', index= None)

In [621]:
import os

# folder path
dir_path = './Label-CurriculumVitae/506/'
count = 0
# Iterate directory
for path in os.listdir(dir_path):
    # check if current path is a file
    if os.path.isfile(os.path.join(dir_path, path)):
        count += 1
print('File count:', count)

File count: 1924


In [259]:
#!zip -r cv_txt_img.zip ./Label-CurriculumVitae